In [21]:
import pandas as pd

pd.set_option("display.max_rows", 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import copy
import itertools
from pprint import pprint


In [22]:
wr.config.s3_endpoint_url = "http://192.168.1.7:8333"
srcBucketName = "multilabel_df"
labelDf = wr.s3.read_parquet(
    path=f"s3://{srcBucketName}/",
    dataset=True
)
labelDf

,vision_bonnet,vision_bumper_front,vision_grille,vision_headlamp_rh,vision_headlamp_lh,vision_door_front_lh,vision_door_front_rh,vision_engine,vision_bumper_rear,vision_misc,...,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear,CaseID,Circumstances_of_Accident
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,12657878,Collision- Head to Rear (Insured Hit TP)
1,1,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,12657888,Lost control- Overturned
2,1,1,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657890,
3,1,1,1,0,0,0,1,1,0,1,...,1,0,0,0,0,0,0,0,12657900,Collided into animal
4,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657903,Collision- Head to Rear (Insured Hit TP)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621787,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10147236,
621788,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10936139,Collision- Head to Rear (TP Hit Insured)
621789,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174259,
621790,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174498,


In [23]:
mappingDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/tmp/complete_view_mapping.csv")

In [24]:
mappingDf["lvl_3_desc"] = mappingDf["lvl_3_desc"].str.strip()

In [25]:
view = ["front_view", "rear_view", "front_view_left", "front_view_right", "rear_view_left", "rear_view_right"]
viewToPart = dict()
for v in view:
    partsInView = mappingDf[mappingDf[v] == 1]["lvl_3_desc"].unique().tolist()
    viewToPart[v] = ["vision_" + x.replace(" ", "_") for x in partsInView]
pprint(viewToPart)
    

{'front_view': ['vision_bonnet',
                'vision_bumper_front',
                'vision_engine',
                'vision_front_panel',
                'vision_grille',
                'vision_headlamp_lh',
                'vision_headlamp_rh',
                'vision_misc',
                'vision_non_external',
                'vision_windscreen_front'],
 'front_view_left': ['vision_bonnet',
                     'vision_bumper_front',
                     'vision_door_front_lh',
                     'vision_door_rear_lh',
                     'vision_engine',
                     'vision_fender_front_lh',
                     'vision_front_panel',
                     'vision_headlamp_lh',
                     'vision_non_external',
                     'vision_misc',
                     'vision_wheel',
                     'vision_windscreen_front'],
 'front_view_right': ['vision_bonnet',
                      'vision_bumper_front',
                      'vision_door_front_r

In [26]:
vehicleType= "Saloon - 4 Dr"
localFileMetadata = f"/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs_metadata/{vehicleType}.parquet"

In [27]:
localFilesDf = pd.read_parquet(localFileMetadata)
localFilesDf["StdDocDesc"] = localFilesDf["StdDocDesc"].str.replace(" ", "_").str.lower()
localFilesDf.sort_values(by="CaseID", inplace=True)
localFilesDf

,CaseID,iDOCID,StdDocDesc,filename,url
6,10000008,437283237.0,front_view,10000008_437283237.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
910895,10000008,437283239.0,front_view_right,10000008_437283239.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
1753935,10000008,437283247.0,rear_view_left,10000008_437283247.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
2375727,10000008,437283248.0,rear_view_right,10000008_437283248.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
1532644,10000008,437283245.0,rear_view,10000008_437283245.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
...,...,...,...,...,...
2288584,13081272,595315679.0,rear_view_left,13081272_595315679.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13081...
247887,13081272,595315614.0,front_view,13081272_595315614.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13081...
2910374,13081272,595315672.0,rear_view_right,13081272_595315672.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13081...
1722827,13081272,595315659.0,rear_view,13081272_595315659.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13081...


In [28]:
viewDfMap = dict()
for v, parts in viewToPart.items():
    tempDf = labelDf[parts + ["CaseID"]]
    tempDf["view"] = v
    viewFilesDf = localFilesDf[localFilesDf["StdDocDesc"] == v]
    viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
    tempDf = tempDf.merge(viewFilesDf[["filename", "CaseID"]], on="CaseID")
    viewDfMap[v] = tempDf


/tmp/ipykernel_1217241/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDf["view"] = v
/tmp/ipykernel_1217241/903975900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
/tmp/ipykernel_1217241/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [29]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    wr.s3.to_csv(
        df=df,
    path=f"s3://imgs_labels/{imgLabelFilename}",

    )

In [30]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    df = wr.s3.read_csv(
    path=f"s3://imgs_labels/{imgLabelFilename}",

    )
    print(df)

       Unnamed: 0  vision_bonnet  vision_bumper_front  vision_engine  \
0               0              1                    1              1   
1               1              1                    1              1   
2               2              1                    1              1   
3               3              1                    1              1   
4               4              1                    1              1   
...           ...            ...                  ...            ...   
14372       14372              0                    0              0   
14373       14373              0                    0              0   
14374       14374              0                    0              0   
14375       14375              0                    0              0   
14376       14376              0                    0              0   

       vision_front_panel  vision_grille  vision_headlamp_lh  \
0                       1              1                   1   
1      

: 